In [1]:
from datasets import load_dataset

import json
import numpy as np
import pandas as pd

In [2]:
dataset_wiki = load_dataset("wikitext", 'wikitext-103-raw-v1')
dataset_cnn = load_dataset("cnn_dailymail", '3.0.0')
dataset_tweet = load_dataset("tweets_hate_speech_detection")

datasets: list = dataset_cnn['train']['article'] + dataset_wiki['test']['text'] + dataset_tweet['train']['tweet']

Reusing dataset wikitext (/home/philko/.cache/huggingface/datasets/wikitext/wikitext-103-raw-v1/1.0.0/47c57a6745aa5ce8e16a5355aaa4039e3aa90d1adad87cef1ad4e0f29e74ac91)
Reusing dataset cnn_dailymail (/home/philko/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/0128610a44e10f25b4af6689441c72af86205282d26399642f7db38fa7535602)
Using custom data configuration default
Reusing dataset tweets_hate_speech_detection (/home/philko/.cache/huggingface/datasets/tweets_hate_speech_detection/default/0.0.0/c32a982d8b2d6233065d820ac655454174f8aaa8faddc74979cf793486acd3b0)


In [3]:
non_raps: pd.DataFrame = pd.DataFrame.from_dict({'text': datasets, 'n': list(map(lambda e: len(e) if len(e) > 150 else None, datasets))})
non_raps = non_raps.dropna()
del datasets

Load lyrics (pd.DataFrame with one lyrics-column)

In [24]:
raps: pd.DataFrame = pd.read_json('Lyrics.json')

In [5]:
raps = raps.rename(columns={'lyrics':'text'})
raps['n'] = raps['text'].apply(lambda row: len(row) if len(row) > 150 else None)
raps = raps.dropna()

# Analyzing character length

In [6]:
np.mean(raps['n'])

2662.064718162839

In [7]:
np.mean(non_raps['n'])

4012.397370452021

Since both datasets are unequal with respect to text length, the non-rap dataset must be truncated.

In [8]:
raps_max = max(raps['n'])
non_raps_clipped = non_raps[non_raps['n'] <= raps_max]
len(non_raps), len(non_raps_clipped)

(288947, 210380)

### Create similar dataset (considering length)

In [9]:
n = int(0.25 * len(raps))
frst_qu = np.percentile(raps['n'], 25)
scnd_qu = np.percentile(raps['n'], 50)
thrd_qu = np.percentile(raps['n'], 75)
frth_qu = raps_max

Sample from quantiles to obtain a similar distributed dataset.

In [10]:
adv_data = pd.concat([non_raps_clipped[non_raps_clipped['n'] < frst_qu].sample(n=n),
    non_raps_clipped[(non_raps_clipped['n'] < scnd_qu) & (non_raps_clipped['n'] >+ frst_qu)].sample(n=n),
    non_raps_clipped[(non_raps_clipped['n'] < thrd_qu) & (non_raps_clipped['n'] >+ scnd_qu)].sample(n=n),
    non_raps_clipped[(non_raps_clipped['n'] < frth_qu) & (non_raps_clipped['n'] >+ thrd_qu)].sample(n=n)])

In [16]:
adv_data['label'] = [0] * len(adv_data)
raps['label'] = [1] * len(raps)
data: pd.DataFrame = pd.concat([adv_data, raps])
data = data.sample(n=len(data), ignore_index=True)
data = data[['text', 'label']]

### Save dataset without length column 

In [18]:
data.to_json('RapNotRap.json')

### Check whether data is similar distributed

In [19]:
np.mean(adv_data['n'])

2771.149953574745

In [22]:
np.std(adv_data['n'])

990.6877696802169

In [20]:
np.mean(raps['n'])

2662.064718162839

In [21]:
np.std(raps['n'])

959.9500546312813

### Looks good ?